In [ ]:
# load packages
import pandas as pd
import os
import json
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
# load datasets voteview.com

# dataset containing information on U.S. legislators
members = pd.read_csv('HSall_members.csv', sep=',')
# dataset containing information on roll calls
roll_calls = pd.read_csv('HSall_rollcalls.csv', sep=',')
# dataset containing information on votes taken by legislators on roll calls
votes = pd.read_csv('HSall_votes.csv', sep=',')

In [ ]:
# remove votes from before 93th congress
votes = votes[votes['congress'] > 92]

# remove votes from Senate
votes = votes[votes['chamber'] == 'House']

In [ ]:
# join votes from 93th congress and members together
votes_members = pd.merge(votes, members, on=['icpsr', 'congress', 'chamber'], how='left')
votes_members

In [ ]:
# add official name of party to dataset
party_name = {100: 'Democratic Party', 200: 'Republican Party', 112: 'Conservative Party', 328: 'Independent', 370: 'Progressive Party',
              537: 'Farmer-Labor Party', 331: 'Independent Republican', 380: 'Socialist Party', 329: 'Independent Democrat', 
              522: 'American Labor Party', 340: 'Populist Party', 347: 'Prohibitionist Party', 356: 'Union Labor Party', 
              213: 'Progressive Republican Party', 402: 'Liberal Party', 354: 'Silver Republican Party', 523: 'American Labor Party (La Guardia)'} 

votes_members['party'] = votes_members['party_code'].map(party_name)

In [ ]:
# add codes of casting to dataset
cast_codes = {0: 'NA member', 1: 'Yea', 2: 'Paired Yea', 3: 'Announced Yea', 4: 'Announced Nay', 5: 'Paired Nay', 6: 'Nay', 
              7: 'Present', 8: 'Present', 9: 'Abstention'}

votes_members['casting'] = votes_members['cast_code'].map(cast_codes)

In [ ]:
# add state full names to dataset
states = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut',
          'DE': 'Delaware', 'DC': 'District of Columbia', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 
          'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 
          'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 
          'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 
          'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 
          'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 
          'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'AS': 'American Samoa', 'MP': 'Northern Mariana Islands', 'GU': 'Guam',
          'PR': 'Puerto Rico', 'VI': 'Virgin Islands'}

votes_members['state'] = votes_members['state_abbrev'].map(states)

In [ ]:
# reorder columns of dataframe to make it better organized
order = ['bioname', 'party', 'state', 'state_abbrev', 'district_code', 'casting', 'congress', 'chamber', 'rollnumber', 
         'icpsr', 'state_icpsr', 'occupancy', 'last_means', 'born', 'died', 'bioguide_id', 'party_code',  'cast_code', 'prob', 
         'nominate_dim1', 'nominate_dim2', 'nominate_log_likelihood', 'nominate_geo_mean_probability', 'nominate_number_of_votes', 
         'nominate_number_of_errors', 'conditional', 'nokken_poole_dim1', 'nokken_poole_dim2'] 

votes_members = votes_members[order]

In [ ]:
# save dataframe containing all information on member and votes. rows are member's votes.
with open("votes_members_full.tsv", "w+") as f:
    votes_members.to_csv(f, sep="\t", index=None)

# MEMBERS

In [ ]:
# load in member votes dataset
df_members = pd.read_csv('votes_members_full.tsv', sep='\t')

# only keep important columns
cols = ['bioname','congress', 'party', 'chamber', 'icpsr', 'state', 'district_code', 'bioguide_id']
df_members = df_members[cols]

# remove duplicates to keep only members per congress
df_members = df_members.drop_duplicates()

# create id based on name and party combination
df_members['nameparty_id'] = 'np_' + (df_members.groupby(['bioname', 'party']).ngroup() + 1).astype(str).str[:-2]

# create id based on name 
df_members['name_id'] = 'n_' + (df_members.groupby(['bioname']).ngroup() + 1).astype(str).str[:-2]

# create id based on name, party, and congress
df_members['member_id'] = 'm_' + (df_members.groupby(['bioname', 'party', 'congress']).ngroup() + 1).astype(str).str[:-2]

# save final members dataset
with open("df_members_fv.tsv", "w+") as f:
    df_members.to_csv(f, sep="\t", index=None)

# BILLS

In [ ]:
# add year column to roll call dataframe
roll_calls['year'] = roll_calls['date'].astype(str).str[:4].astype(int)

# remove roll calls from before 93th congress
df_bills = roll_calls[roll_calls['congress'] > 92]

# remove roll calls from the Senate
df_bills = df_bills[df_bills['chamber'] == 'House']

# take important columns from dataframe and create subdataframe that is only about bills 
bill_order = ['congress', 'chamber', 'rollnumber', 'clerk_rollnumber', 'bill_number', 'year', 'vote_result', 'yea_count', 'nay_count', 
              'vote_desc', 'vote_question', 'dtl_desc']
              
df_bills = df_bills[bill_order]


In [ ]:
# Problem: bill numbers from one dataset contained false letter combinations, leading to trouble merging with new dataset 

# lowercase bill number
df_bills['bill_number'] = df_bills['bill_number'].str.lower()

# divide bill number into number and letters
df_bills['bill'] = df_bills['bill_number'].str.replace(r'\d+', '', regex=True)
df_bills['number'] = df_bills['bill_number'].str.replace(r'\D', '', regex=True)

# replace wrong letter combinations by correct combinations
replace_dict = {
    'hr': 'hr',
    'hres': 'hres',
    's': 's',
    'hconres': 'hconres',
    'hjres': 'hjres',
    'hre': 'hres',
    'hjr': 'hjres',
    'hcr': 'hconres',
    'sjres': 'sjres',
    'hconr': 'hconres',
    'sconres': 'sconres',
    'sjr': 'sjres',
    'scr': 'sconres',
    'hcon': 'hconres',
    'hjre': 'hjres',
    'hcre': 'hconres',
    'sjre': 'sjres',
    'hcres': 'hconres',
    'sconr': 'sconres',
    'scres': 'sconres',
    'scon': 'sconres',
    'scre': 'sconres',
    'sres': 'sres',
    'h': 'hr',
    'hj': 'hjres',
    'hhr': 'hr'
}
df_bills['bill_1'] = df_bills['bill'].map(replace_dict)

# create new correct bill numbers
df_bills['bill_number_1'] = df_bills['bill_1'] + df_bills['number']

# drop old and unneeded columns
df_bills.drop(columns=['bill', 'number', 'bill_1', 'bill_number'], inplace=True)

# change column name and add column with bill type
df_bills['bill'] = df_bills['bill_number_1'].str.replace(r'\d+', '', regex=True)
df_bills.rename(columns={'bill_number_1': 'bill_number'}, inplace=True)

## Adding extra bill information
Topic modelling on the information present in the voteview.org dataset did not perform well. 

We therefore decided to include information on the bill from ProPublica. 

In [ ]:
# function to scrape summary information from json and xml files
def get_summary(congress, bill, bill_number):
    try:
        # find json file per congress, bill type, and bill number
        file_path = f"congress/{congress}/{bill}/{bill_number}/data.json"
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                data = json.load(f)
                # find summary in data and get text
                summary = data['summary']['text']
            return summary
        else:
            # if no json file, find xml file per congress, bill type, and bill number
            xml_file_path = f"congress/{congress}/{bill}/{bill_number}/fdsys_billstatus.xml"
            if os.path.exists(xml_file_path):
                tree = ET.parse(xml_file_path)
                root = tree.getroot()
                # find summaries in root
                summaries = root.find('.//summaries')
                if summaries is not None:
                    # summaries often contain multiple (similar) texts
                    text_list = []
                    for summary in summaries.findall('.//summary'):
                        # get text from each summary and append into list
                        text_element = summary.find('.//text')
                        if text_element is not None:
                            text = text_element.text.strip()
                            text_list.append(text)
                        else:
                            return None
                    # remove duplicate summaries
                    text_list = list(set(text_list))
                    filtered_list = [item for item in text_list if item != '']
                    # return summary as string 
                    result_string = ', '.join(filtered_list)
                    return result_string
                else:
                    return None
            else:
                return None
    except Exception as e:
        print(e)
        return None
    
# add summary to new column based on congress, bill type, and bill number
df_bills['summary'] = df_bills.apply(lambda row: get_summary(row['congress'], row['bill'], row['bill_number']), axis=1)

In [ ]:
# fill rows with no summary with empty string
df_bills['summary'] = df_bills['summary'].fillna('')

# Problem: from xml files the html text was also copied
# remove html text
df_bills['summary'] = df_bills['summary'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

# save new dataframe with all rows containing summary information
with open("df_bills.tsv", "w+") as f:
    df_bills.to_csv(f, sep="\t", index=None)

## Adding topic information 
To be able to test if topic modelling on summary works correctly, we included policy information from ProPublica to dataframe.

In [ ]:
# function to get main policy area of bill (only from xml files)
def get_policy1(congress, bill, bill_number):
    try:
        # find xml file per congress, bill type, and bill number
        xml_file_path = f"congress/{congress}/{bill}/{bill_number}/fdsys_billstatus.xml"
        if os.path.exists(xml_file_path):
            tree = ET.parse(xml_file_path)
            root = tree.getroot()
            # find policy in root
            policy = root.find('.//policyArea')
            if policy is not None:
                # policy can contain multiple (similar) texts
                name_list = []
                # find name of policy
                name = policy.find('name')
                if name is not None:
                    # take text from name and put in list
                    name = name.text.strip()
                    name_list.append(name)
                else:
                    return None
                # remove duplicate policies
                name_list = list(set(name_list))
                filtered_list = [item for item in name_list if item != '']
                # return policy as string
                result_string = ', '.join(filtered_list)
                return result_string
            else:
                return None
        else:
            return None
    except Exception as e:
        print(e)
        return None

# function to get main policy area of bill (only for json files)
def get_policy2(congress, bill, bill_number):
    try:
        # find json file per congress, bill type, and bill number
        file_path = f"congress/{congress}/{bill}/{bill_number}/data.json"
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                data = json.load(f)
                # find top subject in data (equal to policy)
                policy = data['subjects_top_term']
            return policy
        else:
            return None
    except Exception as e:
        print(e)
        return None
        
# add main policy area to new column based on congress, bill type, and bill number
df_bills['policy_1'] = df_bills.apply(lambda row: get_policy1(row['congress'], row['bill'], row['bill_number']), axis=1)

# add main policy area to new column based on congress, bill type, and bill number
df_bills['policy_2'] = df_bills.apply(lambda row: get_policy2(row['congress'], row['bill'], row['bill_number']), axis=1)

In [ ]:
# join policy columns together. if both have a different policy, then combine the two. if both have the same, then take only one of them.
df_bills['policy'] = np.where((~df_bills['policy_1'].isnull()) & (~df_bills['policy_2'].isnull()) & (df_bills['policy_1'] != df_bills['policy_2']),
                                 df_bills['policy_1'].astype(str) + ', ' + df_bills['policy_2'].astype(str),
                                 df_bills['policy_1'].combine_first(df_bills['policy_2']))

# drop old policy columns
df_bills = df_bills.drop(columns=['policy_1', 'policy_2'])

# add bill id
df_bills['bill_id'] = ['b_' + str(i) for i in range(1, len(df_bills) + 1)]

# find bills without summary
no_sum = df_bills[df_bills['summary'] == '']

# find indice of those bills
no_sum_indices = no_sum.index

# remove bills from dataset that have no summary
df_bills = df_bills.drop(no_sum_indices)

# save final bills dataset
with open("df_bills_fv.tsv", "w+") as f:
    df.to_csv(f, sep="\t", index=None)

# VOTES

In [ ]:
# load in member votes dataset
df_member_votes = pd.read_csv('votes_members_full.tsv', sep='\t')

# remove votes without bill number
df_member_votes = df_member_votes.dropna(subset=['bill_number'])

# drop not important columns
df_member_votes.drop(columns=['clerk_rollnumber'], inplace=True)
df_member_votes.drop(columns=['district_code'], inplace=True)

# drop duplicate votes
df_member_votes = df_member_votes.drop_duplicates()

# create three dataset from members dataset, each containing own id
m_col = df_members[['bioname', 'party', 'congress', 'member_id']]
n_col = df_members[['bioname', 'name_id']]
np_col = df_members[['bioname', 'party', 'nameparty_id']]

# drop duplicates, thereby only keeping rows with unique id
n_col = n_col.drop_duplicates()
m_col = m_col.drop_duplicates()
np_col = np_col.drop_duplicates()

# add name id to votes dataset
df_member_votes = pd.merge(df_member_votes, n_col, on=['bioname'], how='left')
df_member_votes = df_member_votes.drop_duplicates()

# add name party id to votes dataset
df_member_votes = pd.merge(df_member_votes, np_col, on=['bioname', 'party'], how='left')
df_member_votes = df_member_votes.drop_duplicates()

# add member id to votes dataset
df_member_votes = pd.merge(df_member_votes, m_col, on=['bioname', 'party', 'congress'], how='left')
df_member_votes = df_member_votes.drop_duplicates()

# create dataset from bills dataset only keeping id information
b_col = df_bills[['congress', 'bill_number', 'rollnumber', 'bill_id']]

# drop duplicates, thereby only keeping rows with unique bill id
b_col = b_col.drop_duplicates()

# add bill id to votes dataset
df_member_votes = pd.merge(df_member_votes, b_col, on=['congress', 'bill_number', 'rollnumber'], how='left')

# save final votes dataset
with open("df_member_votes_fv.tsv", "w+") as f:
    df_member_votes.to_csv(f, sep="\t", index=None)

# After LDA Topic Modelling

In [ ]:
# load dataset bills with topic
df_topic = pd.read_csv('df_bills_fv_with_topics_3.tsv', sep='\t')
df_mv = pd.read_csv('df_member_votes_fv.tsv', sep='\t')

# only keep relevant columns from topic dataset
cols = ['bill_id', 'topic']
df_topic = df_topic[cols]

# join votes and topic datasets together
df_mv = pd.merge(df_topic, df_bt, on=['bill_id'], how='left')

# save dataframe with votes and topics, which can be used for NOMINATE and CA methods
with open("df_member_votes_fv_with_topics_3.tsv", "w+") as f:
    df_mv.to_csv(f, sep="\t", index=None)